In [1]:
import sys
sys.path.append("..")

from utils.support import pkl_load
from train.run import MyDataset
from train.versions.augmentation import get_augmentation
from train.versions.model import get_model

from pathlib import Path
from torch.utils.data import DataLoader
import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
cfg = {
    "model_version": 4,
    "model_weights": "../output/models/v-4/fold-1/model.pt",
    
    "augmentation_version": 1,
    
    "device": "cuda",
    "batch_size": 800,
    "n_jobs": 4,
    
    "confidence_ths": 0.3
    ""
}

In [3]:
dct = pkl_load(Path("../output/crossval_split/crossval_dct.pkl"))

df = dct[1]["valid"]
dataset = MyDataset(df, get_augmentation(cfg["augmentation_version"])["valid"])
dataloader = DataLoader(dataset, batch_size=cfg["batch_size"], shuffle=False, num_workers=cfg["n_jobs"])

In [4]:
model = get_model(cfg["model_version"], cfg["model_weights"])
model.to(cfg["device"])
pass

In [5]:
with torch.no_grad():
    model.eval()
    
    pred = np.array([])
    for x, y in dataloader:
        x, y = x.to(cfg["device"]), y.to(cfg["device"])
        p = model.forward(x)
        p = torch.sigmoid(p)
        p = p.to("cpu").numpy()
        pred = np.append(pred, p)
df["pred_proba"] = pred
df["pred"] = (df["pred_proba"] > 0.5).astype(int)

In [ ]:
def preview(df, ncols, size, pth=None):
    nrows = int(len(df) / ncols) + (len(df) % ncols > 0)
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols,
                            figsize=(ncols*size, nrows*size))
    fig.patch.set_facecolor('white')
    for ax in axs.flatten():
        ax.axis('off')
    for ax, (_, row) in zip(axs.flatten(), df.iterrows()):
        img = cv2.imread(str(row["cew_img"]), cv2.IMREAD_GRAYSCALE)
        ax.imshow(img, cmap=plt.get_cmap('gray'))
        ax.set_title(f"True-{row.label} | Pred-{round(row.pred_proba, 3)}")

    plt.tight_layout()
    plt.show()
    if pth:
        plt.savefig(pth)
    plt.close()

In [ ]:
1 - 26 / 800

In [ ]:
preview_df = df[(df["label"] != df["pred"]) & (df["label"] == 1)]
print(f"Total images: {preview_df.shape[0]}")
preview(preview_df, 5, 2)

In [ ]:
preview_df = df[(df["label"] != df["pred"]) & (df["label"] == 0)]
print(f"Total images: {preview_df.shape[0]}")
preview(preview_df, 5, 2)

In [ ]:
preview_df = df[
    (df["pred"] == 1) &
    
    (df["pred"] == df["label"]) &
    (df["pred_proba"] > 0.3) &
    (df["pred_proba"] < 0.7)
]
print(f"Total images: {preview_df.shape[0]}")
preview(preview_df, 5, 2)

In [ ]:
preview_df = df[
    (df["pred"] == 0) &
    
    (df["pred"] == df["label"]) &
    (df["pred_proba"] > 0.3) &
    (df["pred_proba"] < 0.7)
]
print(f"Total images: {preview_df.shape[0]}")
preview(preview_df, 5, 2)